# Introduction to Cross-Validation - Lab

## Introduction

In this lab, you'll be able to practice your cross-validation skills!


## Objectives

You will be able to:

- Perform cross validation on a model to determine optimal model performance
- Compare training and testing errors to determine if model is over or underfitting

## Let's get started

We included the code to pre-process below.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

ames = pd.read_csv('ames.csv')

continuous = ['LotArea', '1stFlrSF', 'GrLivArea', 'SalePrice']
categoricals = ['BldgType', 'KitchenQual', 'SaleType', 'MSZoning', 'Street', 'Neighborhood']

ames_cont = ames[continuous]

# log features
log_names = [f'{column}_log' for column in ames_cont.columns]

ames_log = np.log(ames_cont)
ames_log.columns = log_names

# normalize (subract mean and divide by std)

def normalize(feature):
    return (feature - feature.mean()) / feature.std()

ames_log_norm = ames_log.apply(normalize)

# one hot encode categoricals
ames_ohe = pd.get_dummies(ames[categoricals], prefix=categoricals, drop_first=True)

preprocessed = pd.concat([ames_log_norm, ames_ohe], axis=1)

X = preprocessed.drop('SalePrice_log', axis=1)
y = preprocessed['SalePrice_log']

In [5]:
preprocessed.head()

,LotArea_log,1stFlrSF_log,GrLivArea_log,SalePrice_log,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,KitchenQual_Fa,KitchenQual_Gd,...,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker
0,-0.133185,-0.803295,0.529078,0.559876,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,0.113403,0.418442,-0.381715,0.212692,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0.419917,-0.576363,0.659449,0.733795,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,0.103311,-0.439137,0.541326,-0.437232,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,0.878108,0.112229,1.281751,1.014303,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0


### Train-test split

Perform a train-test split with a test set of 20%.

In [2]:
# Import train_test_split from sklearn.model_selection
from sklearn.model_selection import train_test_split

In [6]:
# Split the data into training and test sets (assign 20% to test set)
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = .2)

In [7]:
# A brief preview of train-test split
print(len(X_train), len(X_test), len(y_train), len(y_test))


1168 292 1168 292


### Fit the model

Fit a linear regression model and apply the model to make predictions on test set

In [8]:
# Your code here
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()

linreg.fit(X_train,y_train)
y_hat_test = linreg.predict(X_test)

In [ ]:
# #answer
# from sklearn.linear_model import LinearRegression
# linreg = LinearRegression()

# linreg.fit(X_train, y_train)
# y_hat_test = linreg.predict(X_test)

### Residuals and MSE

Calculate the residuals and the mean squared error on the test set

In [10]:
from sklearn.metrics import mean_squared_error
test_residuals = y_hat_test - y_test

test_mse = mean_squared_error(y_test,y_hat_test)
test_mse

0.17164809331375086

In [ ]:
# # Your code here
# from sklearn.metrics import mean_squared_error
# test_residuals = y_hat_test - y_test

# test_mse = mean_squared_error(y_test, y_hat_test)
# test_mse

## Cross-Validation: let's build it from scratch!

### Create a cross-validation function

Write a function `kfolds()` that splits a dataset into k evenly sized pieces. If the full dataset is not divisible by k, make the first few folds one larger then later ones.

We want the folds to be a list of subsets of data!

In [17]:
def kfolds(data, k):
    # Force data as pandas DataFrame
    data = pd.DataFrame(data)
    data_size = len(data)
    fold_size = data_size//k
    left_over = data_size%k
    folds = []
    start_obs = 0
    for fold_i in range(1,k+1):    # add 1 to fold size to account for leftovers           
        if fold_i <= left_over:
            fold = data.iloc[start_obs : start_obs+fold_size+1]
            folds.append(fold)
            start_obs += fold_size +1
        else:
            fold = data.iloc[start_obs : start_obs+fold_size+1]
            folds.append(fold)
            start_obs += fold_size
    return folds

In [ ]:
# def kfolds(data, k):
#     # Force data as pandas DataFrame
#     data = pd.DataFrame(data)
#     num_observations = len(data)
#     fold_size = num_observations//k
#     leftovers = num_observations%k
#     folds = []
#     start_obs = 0
#     for fold_n in range(1,k+1):
#         if fold_n <= leftovers:
#             #Fold Size will be 1 larger to account for leftovers
#             fold =  data.iloc[start_obs : start_obs+fold_size+1] 
#             folds.append(fold)
#             start_obs +=  fold_size + 1
#         else:
#             fold =  data.iloc[start_obs : start_obs+fold_size] 
#             folds.append(fold)
#             start_obs +=  fold_size
            
#     return folds 

### Apply it to the Ames Housing data

In [15]:
# Make sure to concatenate the data again
ames_data = pd.concat([X.reset_index(drop=True), y], axis=1)

In [19]:
# Apply kfolds() to ames_data with 5 folds
ames_fold = kfolds(ames_data,5)

### Perform a linear regression for each fold and calculate the training and test error

Perform linear regression on each and calculate the training and test error: 

In [25]:
test_errs = []
train_errs = []
k=5

for n in range(k):
    # Split in train and test for the fold
    train = pd.concat([fold for i, fold in enumerate(ames_fold) if i !=n])
    test = ames_fold[n]

    # Fit a linear regression model
    linreg.fit(X_train, y_train)

    # Evaluate Train and Test errors
    y_hat_train = linreg.predict(X_train)
    y_hat_test = linreg.predict(X_test)
    train_residuals = y_hat_train - y_train
    test_residuals = y_hat_test - y_test
    
    train_errs.append(np.mean(train_residuals.astype(float)**2))
    test_errs.append(np.mean(test_residuals.astype(float)**2))

print(train_errs)
print(test_errs)

[0.16082253371085622, 0.16082253371085622, 0.16082253371085622, 0.16082253371085622, 0.16082253371085622]
[0.17164809331375086, 0.17164809331375086, 0.17164809331375086, 0.17164809331375086, 0.17164809331375086]


In [ ]:
# test_errs = []
# train_errs = []
# k=5

# for n in range(k):
#     # Split in train and test for the fold
#     train = pd.concat([fold for i, fold in enumerate(ames_folds) if i!=n])
#     test = ames_folds[n]
#     # Fit a linear regression model
#     linreg.fit(X_train, y_train)
#     #Evaluate Train and Test Errors
#     y_hat_train = linreg.predict(X_train)
#     y_hat_test = linreg.predict(X_test)
#     train_residuals = y_hat_train - y_train
#     test_residuals = y_hat_test - y_test
#     train_errs.append(np.mean(train_residuals.astype(float)**2))
#     test_errs.append(np.mean(test_residuals.astype(float)**2))
# print(train_errs)
# print(test_errs)

## Cross-Validation using Scikit-Learn

This was a bit of work! Now, let's perform 5-fold cross-validation to get the mean squared error through scikit-learn. Let's have a look at the five individual MSEs and explain what's going on.

In [27]:
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import cross_val_score

mse = make_scorer(mean_squared_error)

cv_5_results = cross_val_score(linreg, X, y, cv=5, scoring=mse)
# Your code here

Next, calculate the mean of the MSE over the 5 cross-validation and compare and contrast with the result from the train-test split case.

In [28]:
cv_5_results.mean()

# Your code here

0.17702834210001123

##  Summary 

Congratulations! You are now familiar with cross-validation and know how to use `cross_val_score()`. Remember that the results obtained from cross-validation are robust and always use it whenever possible! 